In [75]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

In [76]:
def splitData(data, ratio):
    dataLength = len(data)
    numberOfTrain = int(dataLength * ratio)
    test = data.iloc[numberOfTrain:dataLength]
    train = data.iloc[0:numberOfTrain]
 
    return train, test

In [77]:
def getRepeatedValues(column):
    values = []
    for i in range(0, len(column)):
        if column[i] not in values:
            values.append(column[i])
    return values

In [78]:
def oneHotEncoding(data, columnNames):
    valuesDictionary = {}
    for columnName in columnNames:
        column = data[columnName]
        repeatedValues = getRepeatedValues(column)
        valuesDictionary[columnName] = repeatedValues
        for i in range(0, len(repeatedValues)):
            data[columnName] = data[columnName].replace(repeatedValues[i], i)
    return data, valuesDictionary

In [79]:
def decodeOneHotEncoding(data, decodingDictionary):
    for column in decodingDictionary:
        for i in range(0, len(decodingDictionary[column])):
            data[column] = data[column].replace(i, decodingDictionary[column][i])
    return data

In [80]:
def labelDataset(data, columnNames, threshold):
    data['passed exam'] = -1
    for i in range(0, len(data)):
        for column in columnNames:
            if data[column][i] <= threshold:
                data['passed exam'][i] = 1
                break
    data.drop(columnNames, axis=1, inplace=True)
    return data


In [96]:
data = pd.read_csv('data/exams.csv')
columnsToEncode = ['gender','race/ethnicity',	'parental level of education',	'lunch',	'test preparation course']
print(data.head())
data, decodingDictionary = oneHotEncoding(data,columnsToEncode)
data = labelDataset(data, ['math score', 'reading score', 'writing score'], 60)
train, test = splitData(data, 0.2)

   gender race/ethnicity parental level of education         lunch  \
0    male        group A                 high school      standard   
1  female        group D            some high school  free/reduced   
2    male        group E                some college  free/reduced   
3    male        group B                 high school      standard   
4    male        group E          associate's degree      standard   

  test preparation course  math score  reading score  writing score  
0               completed          67             67             63  
1                    none          40             59             55  
2                    none          59             60             50  
3                    none          77             78             68  
4               completed          78             73             68  


In [82]:
def rbfKernel(u, v, sigma=1):
    return np.exp(-np.linalg.norm(u-v)**2/(2*sigma**2))

In [83]:
def linearKernel(u, v):
    return np.dot(u, v)

In [93]:
def f(x, parameters, b, X, Y, kernel):
    sum = 0
    for i in range(len(X)):
        sum += parameters[i] * Y[i] * kernel(X[i], x) - b
    return sum

In [85]:
def wNormSquare(X, Y, parameters, kernel):
    sum = 0
    for i in range(len(X)):
        for j in range(len(X)):
            sum += parameters[i] * parameters[j] * Y[i] * Y[j] * kernel(X[i], X[j])
    return sum

In [94]:
def functionToMinimize(parameters, X, Y, kernel, C):
    sum = 0
    for i in range(len(X)):
        sum += max(1-f(X[i],parameters,parameters[-1],X,Y,kernel), 0)
    sum += C * wNormSquare(X,Y,parameters,kernel)
    return sum
        

parameters to wektor pionowy wszystkich alf i b na końcu

In [87]:
def gradientFunctionToMinimize(parameters, X, Y, kernel, C):
    gradient = np.zeros(len(X)+1)
    for n in range(len(X)):
        firstSum = 0
        secondSum = 0
        for i in range(len(X)):
            firstSum += max((-parameters[n]*Y[n]*kernel(X[n], X[i]))*Y[i], 0)
            secondSum += parameters[n]*Y[n]*Y[i]*kernel(X[n], X[i]) + parameters[i]* Y[i]*Y[n]*kernel(X[i], X[n])
        gradient[n] = firstSum + C * secondSum
    for i in range(len(X)):
        gradient[-1] += max(-parameters[-1]*Y[i],0)
    return gradient

In [88]:
def fastestDescent(alfa, epsilon, startingPoint, gradient,functionToMinimize):
    valuesVector = np.array([startingPoint])
    dimension = valuesVector.shape[1]
    iterations = 1
    bestValue = np.inf
    # bestParameters = []
    gradientNorm = np.linalg.norm(gradient(valuesVector[0]))
    while gradientNorm>epsilon:
        valuesVector = np.vstack((valuesVector,np.zeros(dimension)))
        valuesVector[iterations] = valuesVector[iterations-1] - alfa * gradientNorm
        currentValue = functionToMinimize(valuesVector[iterations])
        if bestValue > currentValue:
            bestParameters = valuesVector[iterations-1] # zmiejszyc training set, zwracac bestParameters
            bestValue = currentValue
        gradientNorm = np.linalg.norm(gradient(valuesVector[iterations]))
        iterations +=1

    return bestParameters, valuesVector, iterations


In [89]:
def svmGradientAdapter(X, Y, kernel, C):
    def svmGradient(parameters):
        return gradientFunctionToMinimize(parameters, X, Y, kernel, C)
    return svmGradient

In [90]:
def svmFunctionToMinimizeAdapter(X, Y, kernel, C):
    def svmFunctionToMinimize(parameters):
        return functionToMinimize(parameters, X, Y, kernel, C)
    return svmFunctionToMinimize

In [97]:
# X = train[['gender','race/ethnicity',	'parental level of education',	'lunch',	'test preparation course']]
X = np.vstack([train['gender'],train['race/ethnicity'],train['parental level of education'],train['lunch'],train['test preparation course']]).T
Y = train['passed exam'].to_numpy()
C = 1
kernel = rbfKernel
# kernel = linearKernel
startingParameters = np.random.uniform(-1,1,len(X)+1)
bestValue, valuesVector, iterations = fastestDescent(0.1, 0.1, startingParameters, svmGradientAdapter(X, Y, kernel, C), svmFunctionToMinimizeAdapter(X, Y, kernel, C))
print(bestValue)
print(svmFunctionToMinimizeAdapter(X, Y, kernel, C)(bestValue))

/tmp/ipykernel_33428/1509009728.py:4: RuntimeWarning: invalid value encountered in scalar multiply
  sum += parameters[i] * Y[i] * kernel(X[i], x) - b
/tmp/ipykernel_33428/1509009728.py:4: RuntimeWarning: invalid value encountered in scalar subtract
  sum += parameters[i] * Y[i] * kernel(X[i], x) - b
/tmp/ipykernel_33428/206866904.py:5: RuntimeWarning: invalid value encountered in scalar multiply
  sum += parameters[i] * parameters[j] * Y[i] * Y[j] * kernel(X[i], X[j])
/tmp/ipykernel_33428/2434013257.py:7: RuntimeWarning: invalid value encountered in scalar multiply
  firstSum += max((-parameters[n]*Y[n]*kernel(X[n], X[i]))*Y[i], 0)
/tmp/ipykernel_33428/2434013257.py:8: RuntimeWarning: invalid value encountered in scalar multiply
  secondSum += parameters[n]*Y[n]*Y[i]*kernel(X[n], X[i]) + parameters[i]* Y[i]*Y[n]*kernel(X[i], X[n])


[-897.28576463 -897.23884585 -898.62238268 -897.20338799 -896.87421696
 -896.87453741 -897.5532464  -897.20495486 -898.56947415 -898.32337589
 -897.18226512 -897.21738689 -897.08206627 -897.01608066 -897.11745795
 -897.73953205 -898.47090673 -898.26003076 -898.20348146 -897.28692419
 -897.58642854 -896.76436532 -898.40612242 -898.28490354 -898.14493493
 -898.34918787 -898.48192285 -898.3792141  -897.85290697 -898.19678023
 -897.32679757 -897.23218644 -897.14297763 -897.58480755 -897.88075044
 -898.53920512 -896.94478874 -897.53437486 -897.17368541 -897.54978556
 -896.98713033 -897.98333857 -898.10204716 -898.29028238 -897.67844382
 -898.63106588 -897.41125949 -898.23513149 -897.79575946 -897.55837353
 -898.67904439 -896.96766396 -896.98223849 -897.7842484  -898.72536329
 -897.84728774 -898.4788717  -898.14073976 -897.98633017 -896.94755735
 -898.30852869 -898.18913236 -897.52287143 -897.26066556 -898.16166314
 -897.54618743 -897.71755768 -898.67552721 -898.42396249 -898.33421905
 -897.